# Equilibrium Solver #
The Equilibrium solver is designed to take a reaction network as input and solve for a correct equilibrium solution. It does this by writing the system as a system of equations. Solving these equations can give the expected concentrations of all species at Equilibrium.

It is useful to have this capability because we want to compare the results of our simulations to equilibrium. This allows for the detection of traps and other interesting kinetic effects. 

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../")

In [2]:
from steric_free_simulator import VectorizedRxnNet, ReactionNetwork, EquilibriumSolver

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x7f164b26f290>


As usual, we can start by loading up the reaction network

In [3]:
base_input = '../input_files/trimer.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()

['default_assoc', 1.0]
['A']
100.0
['B']
100.0
['C']
100.0
Parsing rule...
SPLIT_01:  ['A(a)+B(a)', 'A(a!1).B(a!1)']
['A', 'B', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['A(b)+C(b)', 'A(b!1).C(b!1)']
['A', 'C', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['B(b)+C(a)', 'B(b!1).C(a!1)']
['B', 'C', '']
GGGGGGGGGgg
Node-1 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f16481ee7d0>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})
Node-2 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f16481ee7d0>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})
-----
{'A'}
{'A'}
set()
Steric hindrance detected
Node-1 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f16481ee7d0>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})
Node-2 :  (1, {'struct': <networkx.classes.graph.Graph object at 0x7f15af6901d0>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})
-----
{'A'}
{'B'}
{'A'}
False
Orig edges:  []
Nextn edge

A minor annoyance is that the reaction network need the association constants to be resolved, which normally happens at simulation time. We can work around this by generated the vectorized network then writing it back to the normal reaction network.

In [4]:
vec_rn = VectorizedRxnNet(rn)
vec_rn.update_reaction_net(rn)

A
Reactant Sets:
B
Reactant Sets:
C
Reactant Sets:
AB
Reactant Sets:
(0, 1)
AC
Reactant Sets:
(0, 2)
BC
Reactant Sets:
(1, 2)
ABC
Reactant Sets:
(2, 3)
(1, 4)
(0, 5)
Before:  tensor([[-1., -1.,  0.,  0.,  0., -1.,  1.,  1., -0., -0., -0.,  1.],
        [-1.,  0., -1., -1.,  0.,  0.,  1., -0.,  1.,  1., -0., -0.],
        [ 0., -1., -1.,  0., -1.,  0., -0.,  1.,  1., -0.,  1., -0.],
        [ 1.,  0.,  0.,  0., -1.,  0., -1., -0., -0., -0.,  1., -0.],
        [ 0.,  1.,  0., -1.,  0.,  0., -0., -1., -0.,  1., -0., -0.],
        [ 0.,  0.,  1.,  0.,  0., -1., -0., -0., -1., -0., -0.,  1.],
        [ 0.,  0.,  0.,  1.,  1.,  1., -0., -0., -0., -1., -1., -1.]],
       dtype=torch.float64)
Shifting to device:  cpu


Now we will initialize a equilibrium solver object on the reaction network. The constructor will convert the network to a list of polynomial equations and constraints defining the system at equilibrium. For simlisty all interactions are written as there own equation, all simplification is left to the sympy engine. 

Sympy is an open-source module that allows python programs to do symbollic math, similar to closed-source tools like Mathematica. 

In [6]:
eq = EquilibriumSolver(rn)

Off rates:  335.4626279025117
Off rates:  335.4626279025117
Off rates:  335.4626279025117
Off rates:  0.11253517471925906
Off rates:  0.11253517471925906
Off rates:  0.11253517471925906


In order to solve the system of equations, we call EquilbriumSolver's `solve` method. Internally, this call `sympy.nsolve` which uses a variety of numeric methods to find a solution. Since the solver is sensitive to initialization, if a solution is not found a random restart is preformed up to a set number of times.

The result will be a vector with copy numbers for each species. Since this system is a trimer, the first three indices  are the equilibrium monomer subunit concentrations, and as always the last index in the vector is the equilibrium concentration of the complete complex. The other values are equilibrium concentrations of various intermediates. 

In [7]:
sol = eq.solve()
sol

Matrix([
[ 14.69282],
[ 14.69282],
[ 14.69282],
[0.6435264],
[0.6435264],
[0.6435264],
[ 84.02012]])

We can now easily calculate the expected equilibrium complet complex yield using our definition of yield.

In [10]:
print(f"Equilibrium expected yield: {100 * sol[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers])}%")

Equilibrium expected yield: 84.0201245117187%
